In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

# Create a list of Mol objects
amino_acid_supplier = Chem.SDMolSupplier('data/amino_acids/amino_acids-nat20.sdf', sanitize=False, removeHs=False, strictParsing=False)

# Set flags for a limited sanitization (full sanitization results in bad aromaticity detection).
s_flags = Chem.rdmolops.SanitizeFlags.SANITIZE_CLEANUP|Chem.rdmolops.SanitizeFlags.SANITIZE_FINDRADICALS|Chem.rdmolops.SanitizeFlags.SANITIZE_CLEANUPCHIRALITY|Chem.rdmolops.SanitizeFlags.SANITIZE_PROPERTIES|Chem.rdmolops.SanitizeFlags.SANITIZE_ADJUSTHS|Chem.rdmolops.SanitizeFlags.SANITIZE_SETHYBRIDIZATION

# Make a list of the molecule names, instantiated Mol objects and sanitize using the flags set
mol_list = []
amino_acids = []
for mol in amino_acid_supplier:
    mol_name = mol.GetProp('_Name')
    mol_list += [mol_name]
    newmol = Chem.Mol(mol)    # See "Creating, Visualizing, ..." for why this is necessary
    amino_acids += [newmol]
    Chem.SanitizeMol(mol, sanitizeOps=s_flags, catchErrors=True)

In [ ]:
def get_inchi(mol):
    inchi_plus = Chem.rdinchi.MolToInchi(mol)
    inchi = inchi_plus[0]
    inchi_key = Chem.rdinchi.InchiToInchiKey(inchi)
    
    return inchi_key, inchi

inchi_list = []
inchi_keys = []
for i, mol in enumerate(amino_acids):    
    key, inchi = get_inchi(mol)
    inchi_list += inchi
    inchi_keys += key
    print(f"{mol_list[i]}:\t{key}")

In [ ]:
import requests

In [ ]:
baseurl = 'https://api.rsc.org/compounds/v1/'
url = baseurl + 'filter/inchikey'
header = {'apikey': 'fp50xEeMoQGxZGkP0a6CGXMk8g9IPX3k'}
inp = {
    'inchikey': 'QNAYBMKLOCPYGJ-REOHCLBHSA-N'
}
r = requests.post(url, headers=header, json=inp)

In [ ]:
response = r.json()
print(response)
qid = response['queryId']
print(qid)

In [ ]:
url = baseurl + f"filter/{qid}/results"
r2 = requests.get(url, headers=header)

In [ ]:
r2.json()

In [ ]:
url = baseurl + 'records/5735/externalreferences'
r3 = requests.get(url, headers=header)

In [ ]:
r3.status_code

In [ ]:
r3.json()